## Cell 1: Imports and Environment Detection

In [1]:
#!/usr/bin/env python3
"""Map processing notebook - converts PR:BF2 heightmaps to JSON format and DDS minimaps to PNG."""

import os
import sys
import zipfile
import struct
import json
import subprocess
import re
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple

# NumPy for array operations
try:
    import numpy as np
    print("NumPy loaded")
except ImportError:
    print("Installing NumPy...")
    %pip install -q numpy
    import numpy as np
    print("NumPy installed and loaded")

# Pillow for DDS to PNG conversion
try:
    from PIL import Image
    print("Pillow loaded")
except ImportError:
    print("Installing Pillow...")
    %pip install -q Pillow
    from PIL import Image
    print("Pillow installed and loaded")

# Detect environment
try:
    IN_COLAB = 'google.colab' in str(get_ipython())
except:
    IN_COLAB = False

print(f"\n{'='*70}")
print(f"Environment: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")
print(f"Python: {sys.version.split()[0]}")
print(f"NumPy: {np.__version__}")
try:
    print(f"Pillow: {Image.__version__}")
except:
    print(f"Pillow: (version detection unavailable)")
print(f"{'='*70}\n")

# Verify repository structure
repo_root = Path.cwd()
if not (repo_root / 'raw_map_data').exists():
    # Try parent directory (in case running from processor/)
    if (repo_root.parent / 'raw_map_data').exists():
        repo_root = repo_root.parent
        os.chdir(repo_root)
    else:
        print("⚠ WARNING: raw_map_data/ directory not found!")
        print("   Make sure you cloned the repository and are in the correct directory.")
        print(f"   Current directory: {repo_root}")

print(f"Repository root: {repo_root}")
print(f"Setup complete")

NumPy loaded
Pillow loaded

Environment: Local Jupyter
Python: 3.14.0
NumPy: 2.3.5
Pillow: 12.0.0

Repository root: c:\Projects\Project_Reality-Mortar-Calculator
Setup complete


## Cell 2: Helper Functions

In [2]:
def parse_terrain_con(content: str) -> Optional[float]:
    """Parse terrain.con file to extract height scale.
    
    Args:
        content: Content of terrain.con file
        
    Returns:
        Height scale in meters, or None if not found
    """
    # Look for: HeightmapCluster.setHeightScale X
    match = re.search(r'HeightmapCluster\.setHeightScale\s+(\d+\.?\d*)', content, re.IGNORECASE)
    if match:
        return float(match.group(1))
    return None


def parse_init_con(content: str) -> Optional[int]:
    """Parse init.con file to extract map size.
    
    Args:
        content: Content of init.con file
        
    Returns:
        Map size in meters, or None if not found
    """
    # Look for: heightmapCluster.create X Y Z
    match = re.search(r'heightmapCluster\.create\s+(\d+)\s+(\d+)', content, re.IGNORECASE)
    if match:
        return int(match.group(1))  # First value is map size
    return None


def extract_heightmap_raw(server_zip_path: Path) -> Optional[np.ndarray]:
    """Extract and parse HeightmapPrimary.raw from server.zip.
    
    Args:
        server_zip_path: Path to server.zip file
        
    Returns:
        2D NumPy array of uint16 values, or None if extraction failed
    """
    try:
        with zipfile.ZipFile(server_zip_path, 'r') as zf:
            # Find heightmapprimary.raw (case-insensitive)
            heightmap_file = None
            for name in zf.namelist():
                if 'heightmapprimary.raw' in name.lower():
                    heightmap_file = name
                    break
            
            if not heightmap_file:
                return None
            
            # Read raw bytes
            raw_data = zf.read(heightmap_file)
            
            # Determine resolution from file size
            # 16-bit = 2 bytes per pixel
            num_pixels = len(raw_data) // 2
            resolution = int(np.sqrt(num_pixels))
            
            # Parse as 16-bit unsigned integers (little-endian)
            heightmap_1d = np.frombuffer(raw_data, dtype='<u2')  # '<u2' = little-endian uint16
            
            # Reshape to 2D array
            heightmap_2d = heightmap_1d.reshape((resolution, resolution))
            
            return heightmap_2d
            
    except Exception as e:
        print(f"  Error extracting heightmap: {e}")
        return None


def extract_config_files(server_zip_path: Path) -> Tuple[Optional[str], Optional[str]]:
    """Extract init.con and terrain.con from server.zip.
    
    Args:
        server_zip_path: Path to server.zip file
        
    Returns:
        Tuple of (init_con_content, terrain_con_content)
    """
    init_con = None
    terrain_con = None
    
    try:
        with zipfile.ZipFile(server_zip_path, 'r') as zf:
            for name in zf.namelist():
                if name.endswith('init.con'):
                    init_con = zf.read(name).decode('utf-8', errors='ignore')
                elif name.endswith('terrain.con'):
                    terrain_con = zf.read(name).decode('utf-8', errors='ignore')
    except Exception as e:
        print(f"  Error extracting config files: {e}")
    
    return init_con, terrain_con


def extract_and_convert_minimap(client_zip_path: Path, map_output_dir: Path, map_name: str) -> Optional[Dict]:
    """Extract DDS minimap from client.zip and convert to PNG.
    
    Args:
        client_zip_path: Path to client.zip file
        map_output_dir: Output directory for this map
        map_name: Name of the map
        
    Returns:
        Dict with minimap metadata, or None if extraction failed
    """
    try:
        with zipfile.ZipFile(client_zip_path, 'r') as zf:
            # Find ingamemap.dds in hud/minimap/ directory
            dds_file = None
            for name in zf.namelist():
                if 'hud/minimap/ingamemap.dds' in name.lower():
                    dds_file = name
                    break
            
            if not dds_file:
                return None
            
            selected_dds = dds_file
            
            # Extract DDS to temporary location
            temp_dds_path = map_output_dir / 'temp_minimap.dds'
            with open(temp_dds_path, 'wb') as f:
                f.write(zf.read(selected_dds))
            
            # Convert DDS to PNG using Pillow
            try:
                img = Image.open(temp_dds_path)
                png_path = map_output_dir / 'minimap.png'
                img.save(png_path, 'PNG')
                
                # Clean up temp file
                temp_dds_path.unlink()
                
                # Validate PNG
                png_size_kb = png_path.stat().st_size / 1024
                width, height = img.size
                
                # Warn if file seems too small (likely corrupted)
                if png_size_kb < 100:
                    print(f"  ⚠ WARNING: PNG size ({png_size_kb:.1f} KB) unusually small, may be corrupted")
                
                # Warn if dimensions not power of 2
                if width != height or width not in [1024, 2048, 4096]:
                    print(f"  ⚠ WARNING: PNG dimensions ({width}x{height}) not standard (expected 1024, 2048, or 4096)")
                
                return {
                    'source_file': selected_dds,
                    'resolution': f"{width}x{height}",
                    'file_size_kb': round(png_size_kb, 1),
                    'converted_at': datetime.utcnow().isoformat() + 'Z'
                }
                
            except Exception as e:
                print(f"  ⚠ Failed to convert DDS to PNG: {e}")
                # Clean up temp file if it exists
                if temp_dds_path.exists():
                    temp_dds_path.unlink()
                return None
                
    except Exception as e:
        print(f"  Error extracting minimap: {e}")
        return None


def convert_heightmap_to_json(heightmap: np.ndarray, output_path: Path) -> bool:
    """Convert heightmap array to JSON format.
    
    Args:
        heightmap: 2D NumPy array of uint16 values
        output_path: Path to output JSON file
        
    Returns:
        True if successful, False otherwise
    """
    try:
        resolution = heightmap.shape[0]
        
        # Flatten to 1D array (row-major order)
        heightmap_flat = heightmap.flatten().tolist()
        
        # Create JSON structure
        data = {
            'resolution': resolution,
            'width': resolution,
            'height': resolution,
            'format': 'uint16',
            'data': heightmap_flat,
            'compression': 'none'
        }
        
        # Write JSON
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, separators=(',', ':'))  # Compact format
        
        return True
        
    except Exception as e:
        print(f"  Error converting to JSON: {e}")
        return False


def generate_metadata(map_name: str, heightmap: np.ndarray, 
                     map_size: Optional[int], height_scale: Optional[float],
                     minimap_metadata: Optional[Dict],
                     output_path: Path) -> bool:
    """Generate metadata.json file.
    
    Args:
        map_name: Name of the map
        heightmap: 2D NumPy array
        map_size: Map size in meters (from init.con)
        height_scale: Height scale in meters (from terrain.con)
        minimap_metadata: Minimap metadata dict (or None if no minimap)
        output_path: Path to output JSON file
        
    Returns:
        True if successful, False otherwise
    """
    try:
        resolution = heightmap.shape[0]
        
        # Use defaults if not found
        if map_size is None:
            # Guess from resolution
            map_size = 2048 if resolution == 1025 else 4096
        
        if height_scale is None:
            height_scale = 300  # Default
        
        # Calculate grid scale
        # 13x13 grid system
        grid_scale = map_size / 13
        
        metadata = {
            'map_name': map_name,
            'map_size': map_size,
            'height_scale': height_scale,
            'grid_scale': grid_scale,
            'heightmap_resolution': resolution,
            'processed_at': datetime.utcnow().isoformat() + 'Z',
            'format_version': '1.0'
        }
        
        # Add minimap metadata if present
        if minimap_metadata:
            metadata['minimap'] = minimap_metadata
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        return True
        
    except Exception as e:
        print(f"  Error generating metadata: {e}")
        return False


print("Helper functions loaded")

Helper functions loaded


## Cell 3: Git Configuration (Optional)

**Optional:** Provide Git credentials if you want to automatically commit and push changes.
**Skip this cell** if you prefer to commit manually later.

In [5]:
print("Git Configuration (Optional)\n" + "="*70)
print("Do you want to automatically commit and push changes to GitHub?")
print("If you skip this, you can commit manually later.\n")

# Ask if user wants automatic commit
auto_commit = input("Enable automatic commit/push? (y/n): ").strip().lower()

if auto_commit == 'y':
    print("\nPlease provide your Git credentials:\n")
    
    # Get user input
    GIT_USER_NAME = input("Git user name (e.g., 'John Doe'): ").strip()
    GIT_USER_EMAIL = input("Git user email (e.g., 'john@example.com'): ").strip()
    GITHUB_TOKEN = input("GitHub Personal Access Token: ").strip()
    
    # Validate inputs
    if not all([GIT_USER_NAME, GIT_USER_EMAIL, GITHUB_TOKEN]):
        print("\n⚠ ERROR: All fields are required!")
        print("Automatic commit/push will be disabled.")
        auto_commit = 'n'
    else:
        # Configure git
        try:
            subprocess.run(['git', 'config', 'user.name', GIT_USER_NAME], check=True, capture_output=True)
            subprocess.run(['git', 'config', 'user.email', GIT_USER_EMAIL], check=True, capture_output=True)
            
            print("\nGit configured successfully")
            print(f"  User: {GIT_USER_NAME} <{GIT_USER_EMAIL}>")
            print(f"  Token: {'*' * len(GITHUB_TOKEN)}")
            
            # Get repository info
            result = subprocess.run(
                ['git', 'config', '--get', 'remote.origin.url'],
                capture_output=True, text=True, check=True
            )
            repo_url = result.stdout.strip()
            print(f"  Repository: {repo_url}")
            print("\n✓ Automatic commit/push enabled")
            
        except subprocess.CalledProcessError as e:
            print(f"\n⚠ ERROR: Git configuration failed")
            print(f"  {e}")
            print("  Make sure you're in a git repository and git is installed.")
            print("Automatic commit/push will be disabled.")
            auto_commit = 'n'
else:
    print("\n✓ Automatic commit/push disabled")
    print("  You can manually commit changes after processing completes.")
    GIT_USER_NAME = None
    GIT_USER_EMAIL = None
    GITHUB_TOKEN = None

Git Configuration (Optional)
Do you want to automatically commit and push changes to GitHub?
If you skip this, you can commit manually later.


✓ Automatic commit/push disabled
  You can manually commit changes after processing completes.


## Cell 4: Load Manifest and Discover Maps

In [6]:
# Load manifest
raw_data_dir = repo_root / 'raw_map_data'
manifest_path = raw_data_dir / 'manifest.json'

if not manifest_path.exists():
    print("⚠ WARNING: manifest.json not found!")
    print(f"  Expected at: {manifest_path}")
    print("  Run processor/collect_maps.py first to collect map data.")
    manifest_data = {'maps': []}
else:
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest_data = json.load(f)
    
    print(f"Loaded manifest")
    print(f"  Total maps: {manifest_data.get('total_maps', 0)}")
    print(f"  With minimaps: {manifest_data.get('maps_with_minimaps', 0)}")
    print(f"  Heightmap-only: {manifest_data.get('maps_heightmap_only', 0)}")
    print(f"  Total size: {manifest_data.get('total_size_mb', 0):.1f} MB")
    print(f"  Collection date: {manifest_data.get('collection_date', 'unknown')}")

# Discover server.zip and client.zip files
map_files = []
for map_info in manifest_data.get('maps', []):
    map_name = map_info['name']
    server_zip = raw_data_dir / map_name / 'server.zip'
    client_zip = raw_data_dir / map_name / 'client.zip'
    
    if server_zip.exists():
        has_client = client_zip.exists() and map_info.get('client_zip') is not None
        map_files.append((map_name, server_zip, client_zip if has_client else None))
    else:
        print(f"⚠ WARNING: Missing {server_zip}")

print(f"\nFound {len(map_files)} maps to process")
maps_with_minimaps = sum(1 for _, _, client in map_files if client)
print(f"  {maps_with_minimaps} with minimap support")
print(f"  {len(map_files) - maps_with_minimaps} heightmap-only")

if len(map_files) == 0:
    print("\n⚠ No files to process!")
    print("  Make sure raw_map_data/ directory contains map folders with server.zip files.")

Loaded manifest
  Total maps: 84
  With minimaps: 84
  Heightmap-only: 0
  Total size: 6504.4 MB
  Collection date: 2025-11-19T16:40:55.270284Z

Found 84 maps to process
  84 with minimap support
  0 heightmap-only


## Cell 5: Processing Loop

This cell processes all maps. Progress will be displayed as processing proceeds.

In [7]:
# Create output directory
processed_dir = repo_root / 'processed_maps'
processed_dir.mkdir(exist_ok=True)

# Processing statistics
stats = {
    'total': len(map_files),
    'processed': 0,
    'errors': 0,
    'error_maps': [],
    'minimaps_converted': 0,
    'minimaps_failed': 0
}

print(f"{'='*70}")
print(f"Processing {stats['total']} maps...")
print(f"{'='*70}\n")

start_time = datetime.now()

for i, (map_name, server_zip, client_zip) in enumerate(map_files, 1):
    print(f"[{i}/{stats['total']}] {map_name}")
    
    try:
        # Create output directory for this map
        map_output_dir = processed_dir / map_name
        map_output_dir.mkdir(exist_ok=True)
        
        # Extract heightmap
        print(f"  Extracting heightmap...")
        heightmap = extract_heightmap_raw(server_zip)
        
        if heightmap is None:
            raise Exception("Failed to extract heightmap")
        
        print(f"  Heightmap: {heightmap.shape[0]}x{heightmap.shape[1]} pixels")
        
        # Extract config files
        print(f"  Extracting config files...")
        init_con, terrain_con = extract_config_files(server_zip)
        
        # Parse config values
        map_size = None
        height_scale = None
        
        if init_con:
            map_size = parse_init_con(init_con)
        
        if terrain_con:
            height_scale = parse_terrain_con(terrain_con)
        
        # Use defaults if not found
        if map_size is None:
            map_size = 2048 if heightmap.shape[0] == 1025 else 4096
            print(f"  ⚠ Map size not found, using default: {map_size}m")
        else:
            print(f"  Map size: {map_size}m")
        
        if height_scale is None:
            height_scale = 300
            print(f"  ⚠ Height scale not found, using default: {height_scale}m")
        else:
            print(f"  Height scale: {height_scale}m")
        
        # Process minimap if client.zip exists
        minimap_metadata = None
        if client_zip:
            print(f"  Extracting and converting minimap...")
            minimap_metadata = extract_and_convert_minimap(client_zip, map_output_dir, map_name)
            
            if minimap_metadata:
                print(f"  ✓ Minimap converted: {minimap_metadata['resolution']}, {minimap_metadata['file_size_kb']:.1f} KB")
                stats['minimaps_converted'] += 1
            else:
                print(f"  ⚠ Minimap conversion failed (continuing with heightmap-only)")
                stats['minimaps_failed'] += 1
        else:
            print(f"  ⚠ No client.zip - heightmap-only mode")
        
        # Convert to JSON
        print(f"  Converting heightmap to JSON...")
        heightmap_json_path = map_output_dir / 'heightmap.json'
        
        if not convert_heightmap_to_json(heightmap, heightmap_json_path):
            raise Exception("Failed to convert heightmap to JSON")
        
        file_size_mb = heightmap_json_path.stat().st_size / (1024 * 1024)
        print(f"  Heightmap JSON: {file_size_mb:.1f} MB")
        
        # Generate metadata
        print(f"  Generating metadata...")
        metadata_path = map_output_dir / 'metadata.json'
        
        if not generate_metadata(map_name, heightmap, map_size, height_scale, minimap_metadata, metadata_path):
            raise Exception("Failed to generate metadata")
        
        print(f"  Metadata JSON created")
        
        stats['processed'] += 1
        print(f"  ✓ {map_name} processed successfully\n")
        
    except Exception as e:
        stats['errors'] += 1
        stats['error_maps'].append(map_name)
        print(f"  ✗ ERROR: {e}\n")

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print(f"{'='*70}")
print(f"Processing complete!")
print(f"  Processed: {stats['processed']}/{stats['total']}")
print(f"  Minimaps converted: {stats['minimaps_converted']}")
print(f"  Minimaps failed: {stats['minimaps_failed']}")
print(f"  Errors: {stats['errors']}")
print(f"  Duration: {duration:.1f} seconds ({duration/60:.1f} minutes)")
print(f"{'='*70}\n")

if stats['error_maps']:
    print("Maps with errors:")
    for map_name in stats['error_maps']:
        print(f"  - {map_name}")

Processing 84 maps...

[1/84] adak
  Extracting heightmap...
  Heightmap: 1025x1025 pixels
  Extracting config files...
  ⚠ Map size not found, using default: 2048m
  ⚠ Height scale not found, using default: 300m
  Extracting and converting minimap...


C:\Users\lenovo\AppData\Local\Temp\ipykernel_22392\3748273807.py:155: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'converted_at': datetime.utcnow().isoformat() + 'Z'


  ✓ Minimap converted: 2048x2048, 3340.0 KB
  Converting heightmap to JSON...
  Heightmap JSON: 4.1 MB
  Generating metadata...
  Metadata JSON created
  ✓ adak processed successfully

[2/84] albasrah_2
  Extracting heightmap...
  Heightmap: 1025x1025 pixels
  Extracting config files...
  ⚠ Map size not found, using default: 2048m
  ⚠ Height scale not found, using default: 300m
  Extracting and converting minimap...
  Heightmap JSON: 4.1 MB
  Generating metadata...
  Metadata JSON created
  ✓ adak processed successfully

[2/84] albasrah_2
  Extracting heightmap...
  Heightmap: 1025x1025 pixels
  Extracting config files...
  ⚠ Map size not found, using default: 2048m
  ⚠ Height scale not found, using default: 300m
  Extracting and converting minimap...


C:\Users\lenovo\AppData\Local\Temp\ipykernel_22392\3748273807.py:245: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'processed_at': datetime.utcnow().isoformat() + 'Z',


  ✓ Minimap converted: 2048x2048, 4746.3 KB
  Converting heightmap to JSON...
  Heightmap JSON: 5.5 MB
  Generating metadata...
  Metadata JSON created
  ✓ albasrah_2 processed successfully

[3/84] andromeda
  Extracting heightmap...
  Heightmap: 513x513 pixels
  Extracting config files...
  ⚠ Map size not found, using default: 4096m
  ⚠ Height scale not found, using default: 300m
  Extracting and converting minimap...
  ✓ Minimap converted: 2048x2048, 290.3 KB
  Converting heightmap to JSON...
  Heightmap JSON: 1.5 MB
  Generating metadata...
  Metadata JSON created
  ✓ andromeda processed successfully

[4/84] asad_khal
  Extracting heightmap...
  Heightmap: 513x513 pixels
  Extracting config files...
  ⚠ Map size not found, using default: 4096m
  ⚠ Height scale not found, using default: 300m
  Extracting and converting minimap...
  ✓ Minimap converted: 2048x2048, 4155.6 KB
  Converting heightmap to JSON...
  Heightmap JSON: 1.5 MB
  Generating metadata...
  Metadata JSON created
  ✓ 

## Cell 6: Git Commit and Push

Automatically commits processed maps and pushes to GitHub.

In [ ]:
if stats['processed'] == 0:
    print("⚠ No maps processed, skipping git operations")
elif auto_commit != 'y':
    print(f"{'='*70}")
    print(f"Manual Commit Instructions")
    print(f"{'='*70}\n")
    print("Automatic commit/push was disabled. To commit changes manually, run:\n")
    commit_date = datetime.now().strftime('%Y-%m-%d')
    minimap_info = f" ({stats['minimaps_converted']} with minimaps)" if stats['minimaps_converted'] > 0 else ""
    print("  git add processed_maps/")
    print(f"  git commit -m 'chore: process maps - {stats['processed']} updated{minimap_info} ({commit_date})'")
    print("  git push origin main")
else:
    print(f"{'='*70}")
    print(f"Git Commit and Push")
    print(f"{'='*70}\n")
    
    try:
        # Pull latest changes first
        print("Pulling latest changes...")
        result = subprocess.run(
            ['git', 'pull', 'origin', 'main'],
            capture_output=True, text=True, cwd=repo_root
        )
        
        if result.returncode == 0:
            print("Pulled latest changes")
        else:
            print(f"⚠ Pull warning: {result.stderr}")
        
        # Stage processed_maps directory
        print("\nStaging files...")
        subprocess.run(
            ['git', 'add', 'processed_maps/'],
            check=True, capture_output=True, cwd=repo_root
        )
        print("Staged processed_maps/")
        
        # Check if there are changes to commit
        result = subprocess.run(
            ['git', 'diff', '--cached', '--name-only'],
            capture_output=True, text=True, cwd=repo_root
        )
        
        changed_files = result.stdout.strip().split('\n')
        num_changed = len([f for f in changed_files if f])
        
        if num_changed == 0:
            print("\n⚠ No changes to commit (maps already up to date)")
        else:
            print(f"\nChanged files: {num_changed}")
            
            # Create commit message with minimap stats
            commit_date = datetime.now().strftime('%Y-%m-%d')
            minimap_info = f" ({stats['minimaps_converted']} with minimaps)" if stats['minimaps_converted'] > 0 else ""
            commit_msg = f"chore: process maps - {stats['processed']} updated{minimap_info} ({commit_date})"
            
            print(f"\nCommitting with message: '{commit_msg}'")
            subprocess.run(
                ['git', 'commit', '-m', commit_msg],
                check=True, capture_output=True, cwd=repo_root
            )
            print("Committed changes")
            
            # Get repository URL and configure with token
            result = subprocess.run(
                ['git', 'config', '--get', 'remote.origin.url'],
                capture_output=True, text=True, check=True, cwd=repo_root
            )
            repo_url = result.stdout.strip()
            
            # Convert to HTTPS URL with token if needed
            if repo_url.startswith('https://'):
                # Remove https://
                repo_url = repo_url.replace('https://', '')
                # Add token
                auth_url = f'https://{GITHUB_TOKEN}@{repo_url}'
            elif repo_url.startswith('git@'):
                # Convert SSH to HTTPS
                repo_url = repo_url.replace('git@github.com:', 'github.com/')
                repo_url = repo_url.replace('.git', '')
                auth_url = f'https://{GITHUB_TOKEN}@{repo_url}.git'
            else:
                auth_url = repo_url
            
            # Push to GitHub
            print("\nPushing to GitHub...")
            result = subprocess.run(
                ['git', 'push', auth_url, 'main'],
                capture_output=True, text=True, cwd=repo_root
            )
            
            if result.returncode == 0:
                print("Successfully pushed to GitHub!")
                print(f"\n{'='*70}")
                print("All done! Maps are now available in your repository.")
                print(f"{'='*70}")
            else:
                print(f"\n✗ Push failed: {result.stderr}")
                print("\nYou may need to manually push changes:")
                print("  git push origin main")
    
    except subprocess.CalledProcessError as e:
        print(f"\n✗ Git operation failed: {e}")
        print(f"\nError output: {e.stderr if hasattr(e, 'stderr') else 'N/A'}")
        print("\nYou can manually commit and push:")
        print("  git add processed_maps/")
        print(f"  git commit -m 'chore: process maps - {stats['processed']} updated ({stats['minimaps_converted']} with minimaps)'")
        print("  git push origin main")

Git Commit and Push

Pulling latest changes...
⚠ Pull warning: error: cannot pull with rebase: You have unstaged changes.
error: Please commit or stash them.


Staging files...
Staged processed_maps/

Changed files: 165

Committing with message: 'chore: process maps - 83 updated (2025-11-19)'
Committed changes

Pushing to GitHub...
Successfully pushed to GitHub!

All done! Maps are now available in your repository.


## Cell 7: Summary

Processing complete! Review the summary below.

In [8]:
print(f"\n{'='*70}")
print("PROCESSING SUMMARY")
print(f"{'='*70}\n")

print(f"Maps processed: {stats['processed']}/{stats['total']}")
print(f"Minimaps converted: {stats['minimaps_converted']}")
print(f"Minimap conversions failed: {stats['minimaps_failed']}")
print(f"Errors: {stats['errors']}")

if stats['error_maps']:
    print(f"\nMaps with errors:")
    for map_name in stats['error_maps']:
        print(f"  - {map_name}")

# Calculate total size
total_size = 0
minimap_count = 0
for map_dir in processed_dir.iterdir():
    if map_dir.is_dir():
        for file in map_dir.iterdir():
            if file.is_file():
                total_size += file.stat().st_size
                if file.name == 'minimap.png':
                    minimap_count += 1

total_size_mb = total_size / (1024 * 1024)
print(f"\nTotal output size: {total_size_mb:.1f} MB")
print(f"Output directory: {processed_dir}")
print(f"Minimap PNG files created: {minimap_count}")

# Get repository URL
try:
    result = subprocess.run(
        ['git', 'config', '--get', 'remote.origin.url'],
        capture_output=True, text=True, cwd=repo_root
    )
    repo_url = result.stdout.strip()
    
    # Convert to web URL
    if repo_url.startswith('git@'):
        web_url = repo_url.replace('git@github.com:', 'https://github.com/').replace('.git', '')
    elif repo_url.startswith('https://'):
        web_url = repo_url.replace('.git', '')
    else:
        web_url = repo_url
    
    print(f"\nRepository: {web_url}")
    print(f"Processed maps: {web_url}/tree/main/processed_maps")
except:
    pass

print(f"\n{'='*70}")
print("NEXT STEPS")
print(f"{'='*70}\n")

if stats['processed'] > 0:
    print("1. ✓ Maps processed successfully")
    if auto_commit == 'y':
        print("2. ✓ Changes committed and pushed to GitHub")
    else:
        print("2. → Commit changes manually (see instructions in Cell 6)")
    print(f"3. ✓ {stats['minimaps_converted']} minimaps converted to PNG")
    print("4. → Test the calculator: run calculator/server.py")
    print("5. → Verify maps and minimaps load correctly in the web interface")
else:
    print("⚠ No maps were processed.")
    print("  Check errors above and verify raw_map_data/ contains server.zip files.")

print(f"\n{'='*70}")
print("Notebook execution complete!")
print(f"{'='*70}")


PROCESSING SUMMARY

Maps processed: 83/84
Minimaps converted: 83
Minimap conversions failed: 0
Errors: 1

Maps with errors:
  - the_falklands

Total output size: 760.1 MB
Output directory: c:\Projects\Project_Reality-Mortar-Calculator\processed_maps
Minimap PNG files created: 83

Repository: https://github.com/zadzanl/Project_Reality-Mortar-Calculator
Processed maps: https://github.com/zadzanl/Project_Reality-Mortar-Calculator/tree/main/processed_maps

NEXT STEPS

1. ✓ Maps processed successfully
2. → Commit changes manually (see instructions in Cell 6)
3. ✓ 83 minimaps converted to PNG
4. → Test the calculator: run calculator/server.py
5. → Verify maps and minimaps load correctly in the web interface

Notebook execution complete!
